In [1]:
import cv2
import time
import tensorflow as tf
import tensorflow.keras.metrics as metrics
import numpy as np
class ArgmaxMeanIOU(metrics.MeanIoU):
    def update_state(self, y_true, y_pred, sample_weight=None):
        return super().update_state(tf.argmax(y_true, axis=-1), tf.argmax(y_pred, axis=-1), sample_weight)

def img_pred(src,model,image_size):
    src = cv2.cvtColor(src, cv2.COLOR_BGR2RGB)
    #cv2_imshow(src)
    image = cv2.resize(src,image_size,interpolation=cv2.INTER_CUBIC)
    #cv2_imshow(src)
    image = image/255
    image =  tf.image.per_image_standardization(image)
    data = tf.convert_to_tensor(image, dtype=tf.float32)
    data = tf.expand_dims(data, axis=0)
    s = time.time()
    pred = model.predict(data)
    e = time.time()
    fps = 1/(e-s)
    seg = tf.argmax(pred[0], axis=-1)
    seg = seg[..., tf.newaxis]
    seg = tf.keras.preprocessing.image.array_to_img(seg)
    #plt.imshow(seg)
    seg = cv2.cvtColor(np.array(seg), cv2.COLOR_BGR2RGB)
    seg = cv2.resize(seg,(src.shape[1],src.shape[0]),interpolation=cv2.INTER_CUBIC)
    result = cv2.addWeighted(np.array(src), 0.6, seg, 0.5, 0, dtype = cv2.CV_8U)
    return src, seg, result, fps

C:\Users\andre\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\andre\anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\andre\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [13]:
import statistics as stat
from tqdm import notebook
import time
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

def gen_seg_vid(cap,out,model,subtitle):
    length = 1000
    progress = notebook.tqdm(total = length)
    AVG_FPS = 0
  #while(cap.isOpened()):
    for _ in range(length):
        ret, frame = cap.read()
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2YUV)
        channels = cv2.split(frame)
        frame = cv2.merge(channels)
        channels[0]
        frame = cv2.cvtColor(frame, cv2.COLOR_YUV2BGR)
        cv2.imshow('frame',cv2.resize(frame,(800,400),interpolation=cv2.INTER_CUBIC))
        cv2.waitKey(200)
        if ret == True:
            total_fps = []
            m = model
            shape = (256,128)
            result = img_pred(frame,m,shape)
            img_seg = result[2]
            fps = result[3]
            total_fps.append(fps)
            if (_ % 5) == 0:
                AVG_FPS = stat.mean(total_fps)
            x = 'AVG_FPS:'
            y = 'FPS: '
            img_seg = cv2.resize(img_seg,(512,256),interpolation=cv2.INTER_CUBIC)
            text = "{}{:.3f}".format(x,AVG_FPS)
            text1 = "{}{:.3f}".format(y,fps)
            font = cv2.FONT_HERSHEY_DUPLEX
            img_out = cv2.cvtColor(img_seg, cv2.COLOR_BGR2RGB)
            cv2.putText(img_out, text , (350,230), font, 0.5, (0, 255, 255), 1, cv2.LINE_AA)
            cv2.putText(img_out, text1 , (350,210), font, 0.5, (0, 255, 255), 1, cv2.LINE_AA)
            cv2.putText(img_out, subtitle , (140,20), font, 0.7, (0, 255, 0), 2, cv2.LINE_AA)
            img_out = cv2.resize(img_out,(1800,1300),interpolation=cv2.INTER_CUBIC)
            
            out.write(img_out)
            progress.update(1)
            if cv2.waitKey(100) & 0xFF == ord('q'):
                break
        else:
            break
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    return AVG_FPS

In [14]:
model = tf.keras.models.load_model('E:/EE/project/FPGA/benchmark_model/model_benchmark.tf',custom_objects={'ArgmaxMeanIOU': ArgmaxMeanIOU}, compile=False)
cap = cv2.VideoCapture('E:\EE\project\FPGA\Driving_data.mp4')
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
out = cv2.VideoWriter('E:/EE/project/FPGA/benchmark_model/City_GPU_Ynorm.mp4',fourcc,20,(1800,1300))

subtitle = 'Benchmark mIoU(92%)'

gen_seg_vid(cap,out,model,subtitle)

  0%|          | 0/1000 [00:00<?, ?it/s]

TypeError: No loop matching the specified signature and casting was found for ufunc true_divide

In [ ]:
src = cv2.imread('E:/EE/project/FPGA/test_02.png')
start = time.time()
result = img_pred(src,model,(256,128))[2]
end = time.time()
fps = 1/(end-start)
print('fps =',fps)
result = cv2.resize(result,(1024,512),interpolation=cv2.INTER_CUBIC)
cv2.imshow("result",result)
cv2.waitKey (0)

In [ ]:
def shadow_removal(frame):
    subtraction = cv2.
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    